# Optimización de costos



In [2]:
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## OR-Tools

**Reglas a considerar**
TIPO A:
- Capacidad 1000000
- Solo se abastece LUNES MARTES VIERNES
- Rate costo: 0.1%

TIPO B:
- Capacidad 1300000
- Solo se abastece LUNES MIÉRCOLES JUEVES
- Rate costo: 0.15%

* No se puede abastecer más de una vez al día
* cantidad abastecida + lo que tiene >= 20%  

Debemos **minizar** los costos de abastecimiento, ello implica también reducir la cantidad de abastecimientos en una semana

In [ ]:
    # model.AddBoolAnd([is_ab[i],is_ab[i]])

### Observaciones

In [50]:
df = pd.read_csv('Datafest2024_Test.csv')
fd = pd.read_csv('Datafest2024_Train.csv')
print(df.shape, fd.shape)

(29400, 7) (222600, 7)


In [48]:
print(df['fecha_transaccion'].min(), df['fecha_transaccion'].max())

20240415 20240526


Fecha Inicio:   15 de abril

Fecha Fin:      26 de mayo

In [51]:
print(fd['fecha_transaccion'].min(), fd['fecha_transaccion'].max())

20230602 20240414


Fecha Inicio:   2 de junio 2023

Fecha Fin:      14 de abril 2024

In [56]:
dates = fd['fecha_transaccion'].unique()
len(dates)

318

Voy a probar con fd que es la data de entrenamiento. Tiene 318 días empieza un viernes 2 de junio y termina un lunes 14 de abril. 

In [57]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222600 entries, 0 to 222599
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   fecha_transaccion  222600 non-null  int64 
 1   codigo_cajero      222600 non-null  int64 
 2   tipo_cajero        222600 non-null  object
 3   saldo_inicial      222600 non-null  int64 
 4   demanda            222600 non-null  int64 
 5   abastecimiento     222600 non-null  int64 
 6   saldo_final        222600 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 11.9+ MB


### PARA SOLO UN CAJERO

In [288]:
###### DATOS ####
CAPACIDAD = 100
SI = 80 # día 20
SF = 30 # día 20
AB = 0
# Dem = 50 # día 20 
################

dias_posibles           = [0,1,0,0,1,1,0,0]
predicciones_demanda    = [0, 12, 10, -10, 10, 24, 10, 10]
abastecimiento = []
saldo_inicial = []
saldo_final = []
is_ab = []

## Creando el modelo
model = cp_model.CpModel()

for i in range(0,8):
    saldo_inicial.append(model.NewIntVar(0,CAPACIDAD,f'si[{i}]'))
    saldo_final.append(model.NewIntVar(0,CAPACIDAD,f'sf[{i}]'))
    abastecimiento.append(model.NewIntVar(0,CAPACIDAD,f'ab[{i}]'))
    is_ab.append(model.NewBoolVar(f'is_ab[{i}]'))


# model.Add(saldo_inicial[0]==SI)
model.Add(saldo_final[0]==SF)
# model.Add(abastecimiento[0]==AB)

# demanda[i] = SI - SF + abastecimiento[i]
for i in range(1,8):
    model.Add(saldo_inicial[i] - saldo_final[i] + abastecimiento[i] == predicciones_demanda[i])
    model.Add(saldo_inicial[i]==saldo_final[i-1])
    model.Add(saldo_final[i]>=int(0.2*CAPACIDAD))
    model.Add(abastecimiento[i]+saldo_inicial[i]<=CAPACIDAD)
    model.Add(is_ab[i]==0).OnlyEnforceIf(dias_posibles[i]==0)
    model.Add(abastecimiento[i]==0).OnlyEnforceIf(is_ab[i].negated())
    model.Add(abastecimiento[i]>0).OnlyEnforceIf(is_ab[i])
    

model.minimize(sum(is_ab))

solver = cp_model.CpSolver()
status = solver.Solve(model)
ans = []
is_ab2=[]
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for i in range(1,8):
        ans.append(solver.Value(abastecimiento[i]))
        is_ab2.append(solver.Value(is_ab[i]))
else: print("no solution")
print(ans)
print(is_ab2)

[56, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]


### PARA CUALQUIER CAJERO

In [75]:
Capacity_A = 1000000
Capacity_B = 1300000
days_A = [1,1,0,0,1,0,0]
days_B = [1,0,1,1,0,0,0]
class Cajero:  
    SF=0
    tipo = 'A'
    predicciones_demanda = []
    
def dfToCajero(row):
    cajero = Cajero()
    cajero.SF = row['SF_0']
    predicciones_demanda = []
    for i in range(7):
        predicciones_demanda.append(row[f'predicted{i+1}'])
    cajero.predicciones_demanda = predicciones_demanda
    cajero.tipo = row['Tipo']   
    
    replenishment = meMame(cajero,row['date'])
    ret = [row['date'], row['Code'],row['Tipo']]
    # replenishment = [1,2,3,4,5,6,7]
    ret.extend(replenishment)
    return pd.Series(ret, index = ['date','Code','Tipo','ab1','ab2','ab3','ab4','ab5','ab6','ab7'])

def meMame(cajero, mydate):
    
    CAPACIDAD = Capacity_A if cajero.tipo == 'A' else Capacity_B
    today = mydate.day_of_week
    dias_posibles = days_A if cajero.tipo == 'A' else days_B
    currentdays = [0]
    ax = [dias_posibles[(i+1)%7] for i in range(today,today+7)]
    currentdays.extend(ax)
    SF = cajero.SF
    predicciones_demanda = [0]
    predicciones_demanda.extend(cajero.predicciones_demanda)
    n_dias = len(predicciones_demanda)
    abastecimiento = []
    saldo_inicial = []
    saldo_final = []
    is_ab = []

    model = cp_model.CpModel()
    for i in range(n_dias):
        saldo_inicial.append(model.NewIntVar(0,CAPACIDAD,f'si[{i}]'))
        saldo_final.append(model.NewIntVar(0,CAPACIDAD,f'sf[{i}]'))
        abastecimiento.append(model.NewIntVar(0,CAPACIDAD,f'ab[{i}]'))
        is_ab.append(model.NewBoolVar(f'is_ab[{i}]'))
    
    # model.Add(saldo_inicial[0]==SI)
    # model.Add(abastecimiento[0]==AB)
    model.Add(saldo_final[0]==SF)
    
    for i in range(1,n_dias):
        model.Add(saldo_inicial[i] - saldo_final[i] + abastecimiento[i] == predicciones_demanda[i]) # formula de la demanda
        model.Add(saldo_inicial[i]==saldo_final[i-1]) # secuencia de los días
        model.Add(saldo_final[i]>=int(CAPACIDAD*0.2)) # mayor al 20%
        model.Add(abastecimiento[i]+saldo_inicial[i]<=CAPACIDAD)
        model.Add(is_ab[i]==0).OnlyEnforceIf(currentdays[i]==0)
        model.Add(abastecimiento[i]==0).OnlyEnforceIf(is_ab[i].negated())
        model.Add(abastecimiento[i]>0).OnlyEnforceIf(is_ab[i])
    
    model.minimize(sum(is_ab))
    # model.minimize(sum(is_ab+abastecimiento))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    ans = []
    is_ab2=[]
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        for i in range(1,n_dias):
            ans.append(solver.Value(abastecimiento[i]))
        return ans
    else: print("no hay solu papu")

    

In [33]:
datos = {
    'code': [10,10],
    'date': ['2024-05-20','2024-05-20'],
    'Tipo': ['A','A'],
    'SF_0': [10,10],
    'predicted1': [12,12],
    'predicted2': [10,10],
    'predicted3': [-10,-10],
    'predicted4': [10,10],
    'predicted5': [24,24],
    'predicted6': [10,10],
    'predicted7': [10,10]
}
df = pd.DataFrame(data=datos)
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
df


,code,date,Tipo,SF_0,predicted1,predicted2,predicted3,predicted4,predicted5,predicted6,predicted7
0,10,2024-05-20,A,10,12,10,-10,10,24,10,10
1,10,2024-05-20,A,10,12,10,-10,10,24,10,10


In [34]:
df_new = df.apply(dfToCajero,axis=1)
df_new

KeyError: 'Code'

In [76]:
sama = pd.read_csv('samuelitoA.csv')
samb = pd.read_csv('samuelitoB.csv')
sama.head()

,Unnamed: 0,2024-05-21,2024-05-22,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,Tipo,SaldoFinal
0,Cajero_302,24252.297205,363307.868056,338985.085169,26963.607273,13534.658594,345122.509288,189094.236590,A,386490.0
1,Cajero_303,23463.923097,348147.575026,327965.631553,26087.096086,13094.684834,333903.545241,182947.313730,A,276150.0
2,Cajero_304,26974.442562,352697.326758,330342.287574,24285.168834,12892.282680,320048.028943,180119.530489,A,387010.0
3,Cajero_305,23043.957648,340200.673351,328606.238966,25620.180175,12860.311615,324899.240427,179672.859140,A,305510.0
4,Cajero_306,23333.042246,347862.005853,332728.577846,25941.583278,13021.643191,328975.075308,181926.840730,A,447680.0


In [77]:
sama.rename(columns={'Unnamed: 0':'Code', 'SaldoFinal':'SF_0', 
                     '2024-05-21':'predicted1',
                     '2024-05-22':'predicted2',
                     '2024-05-23':'predicted3',
                     '2024-05-24':'predicted4',
                     '2024-05-25':'predicted5',
                     '2024-05-26':'predicted6',
                     '2024-05-27':'predicted7',
                     }, inplace=True)
sama['date'] = '2024-05-20'
sama['date'] = pd.to_datetime(sama['date'],format='%Y-%m-%d')
sama = sama.astype({col: np.int32 for col in sama.select_dtypes("number").columns})

sama.head()

,Code,predicted1,predicted2,predicted3,predicted4,predicted5,predicted6,predicted7,Tipo,SF_0,date
0,Cajero_302,24252,363307,338985,26963,13534,345122,189094,A,386490,2024-05-20
1,Cajero_303,23463,348147,327965,26087,13094,333903,182947,A,276150,2024-05-20
2,Cajero_304,26974,352697,330342,24285,12892,320048,180119,A,387010,2024-05-20
3,Cajero_305,23043,340200,328606,25620,12860,324899,179672,A,305510,2024-05-20
4,Cajero_306,23333,347862,332728,25941,13021,328975,181926,A,447680,2024-05-20


In [78]:
df_new = sama.apply(dfToCajero,axis=1)
df_new

,date,Code,Tipo,ab1,ab2,ab3,ab4,ab5,ab6,ab7
0,2024-05-20,Cajero_302,A,540054,0,0,574713,0,0,0
1,2024-05-20,Cajero_303,A,623425,0,0,556031,0,0,0
2,2024-05-20,Cajero_304,A,523003,0,0,537344,0,0,0
3,2024-05-20,Cajero_305,A,586339,0,0,543051,0,0,0
4,2024-05-20,Cajero_306,A,456243,0,0,549863,0,0,0
...,...,...,...,...,...,...,...,...,...,...
394,2024-05-20,Cajero_696,A,563950,0,0,540660,0,0,0
395,2024-05-20,Cajero_697,A,624705,0,0,565073,0,0,0
396,2024-05-20,Cajero_698,A,495540,0,0,540733,0,0,0
397,2024-05-20,Cajero_699,A,597119,0,0,553395,0,0,0


In [65]:
samb.rename(columns={'Unnamed: 0':'Code', 'SaldoFinal':'SF_0', 
                     '2024-05-21':'predicted1',
                     '2024-05-22':'predicted2',
                     '2024-05-23':'predicted3',
                     '2024-05-24':'predicted4',
                     '2024-05-25':'predicted5',
                     '2024-05-26':'predicted6',
                     '2024-05-27':'predicted7',
                     }, inplace=True)
samb['date'] = '2024-05-20'
samb['date'] = pd.to_datetime(sama['date'],format='%Y-%m-%d')
samb = samb.astype({col: np.int32 for col in sama.select_dtypes("number").columns})

sama.head()

,Unnamed: 0,2024-05-21,2024-05-22,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,Tipo,SaldoFinal
0,Cajero_1,8278.907500,337887.311472,355134.890585,6152.151686,22206.759209,373338.301651,170779.200536,B,400400.0
1,Cajero_2,15204.790026,346083.775991,341578.706152,22069.012420,25827.520979,367312.648409,166327.243022,B,383710.0
2,Cajero_3,11138.055033,344350.111150,324073.280917,23725.063191,27597.965823,350535.548624,166524.013486,B,456580.0
3,Cajero_4,15122.518394,344278.560827,341224.461155,17831.157207,27518.877521,349644.298539,165568.951487,B,307010.0
4,Cajero_5,15264.334532,340036.454748,322179.126247,24164.913181,29575.307778,365813.205202,165347.821911,B,308900.0


In [10]:
samb.rename(columns={'Unnamed: 0':'Code', 'SaldoFinal':'SF_0'}, inplace=True)
samb['date'] = pd.to_datetime(samb['date'],format='%Y-%m-%d')
samb.head()

KeyError: 'date'